# Prep AAL Inputs
## Create WSE csv files for the AAL tool

Workflow 

1. Read and join WSE csv output files from s3.  
2. Attribute ground elevations given input point file. 
3. Join attributes to the building attribute files stored on s3 for each 'book'

 - Uniform
 - Uncorrelated
 - Market Basket 
     
     
4. Write input for AAL Tool
5. Create Weights file for Pluvial*

**Fluvial Weights created using MC_BreachSimulator*

Author: Alec Brazeau - abrazeau@dewberry.com

# Set Parameters

In [ ]:
import gdal
gdal.UseExceptions()
import os
import sys; sys.path.append('../core')
import pandas as pd
from zipfile import ZipFile
from s3utils import *
from prep import *

In [ ]:
# # Parameters
# TRI = ""
# wse_dir = "C:\\Users\\Administrator\\Desktop\\probmod-tools\\risk\\outputs\\MeridianHills\\F01\\WSE"
# weights_dir = "C:\\Users\\Administrator\\Desktop\\probmod-tools\\risk\\outputs\\MeridianHills\\F01\\Weights"
# project = 'MeridianHills'
# model = 'F01'

#### Check Paths

In [ ]:
if 'F' in model:
    model_type = 'Fluvial'
else:
    model_type = 'Pluvial'
    
s3point_zip = r's3://pfra/RiskAssessment/{0}/Points/{0}_{1}.zip'.format(project, model)

### Create Weights File

In [ ]:
if model_type == 'Pluvial':
    for i in range(99):
        try:
            s3WeightsPath = 's3://pfra/{0}/Data/Forcing/{1}/Outputs/{0}_{2}_D{3}_Weights{4}.json'.format(project, model_type, model, str(i).zfill(2), TRI)
            print(s3WeightsPath)
            weights = make_weights_df(s3WeightsPath)
            weights.to_csv(os.path.join(weights_dir, '_'.join([project, model, 'weights{}.csv'.format(TRI)])))
            print('Weights written to {}'.format(wse_dir))
            break
        except AttributeError as e:
            print('\nError on {}\n'.format(str(i).zfill(2)))
else:
    weights_file = 's3://pfra/RiskAssessment/{0}/BreachAnalysis/{0}_{1}.xlsx'.format(project, model)
    weights = pd.read_excel(weights_file, sheet_name='Event_Weights', index_col=0 )[['Overall Weight']]
#     weights.to_csv(os.path.join(weights_dir, '_'.join([project, model, 'weights{}.csv'.format(TRI)])))
#     print('Weights written to {}'.format(wse_dir))

## Create the WSE dataframe

In [ ]:
# Search s3 for processed wse's for project/model
s3files = s3List('pfra', '{}/{}'.format(project, model), '', '.csv')
print(len(s3files))

In [ ]:
s3files = [f for f in s3files if not 'BreachLocation' in f]
print(len(s3files))

In [ ]:
# Check to make sure there is a WSE csv for each event
if model_type == 'Pluvial':
    wt_events = weights.event_id.tolist()
    csv_events = [os.path.basename(x).split('{}_{}_'.format(project, model))[-1].replace('.csv', '') for x in s3files]
else:
    wt_events = weights.index.tolist()
    csv_events = [os.path.basename(x).split('{}_{}_'.format(project, model))[-1].replace('.csv', '').split('_')[1] for x in s3files]

for e in wt_events:
    assert e in csv_events, 'WSE CSV file for event {} is missing!'.format(e)

In [ ]:
# Print the missing runs
missing = []
for e in wt_events:
    if not e in csv_events:
        missing.append(e)
print(len(missing), 'Missing events:', missing)

#### Read in CSV's for all points

In [ ]:
wse_only = make_WSE_df(s3files)
wse_only.head(2)

## Pull elevations at each point
### Download and extract points shapefile

In [ ]:
s3download([s3point_zip], wse_dir)
localzip = os.path.join(wse_dir, os.path.basename(s3point_zip))
with ZipFile(localzip, 'r') as zip_ref:
    zip_ref.extractall(localzip.replace('.zip', ''))
points_shapefile = glob(os.path.join(localzip.replace('.zip', ''), '*shp'))[0]
print('Points Shapefile downloaded and extracted to: {}'.format(points_shapefile))

### Retrieve the elevations

In [ ]:
elev = get_elevations(project, model, points_shapefile)
elev.head(2)

### Join Input Data
 - WSE results from events
 - Ground elevation from topo
 - Actuarial data fields (buildings attributes)
 
### Save Final Inputs for AAL Tool to output directory

In [ ]:
attribute_files = s3List('pfra', 'RiskAssessment/{}/Attributes/{}'.format(project, model), '', '.csv')

for f in attribute_files:
    if 'Weights' in f or 'AAL' in f:
        continue
    attrs = load_s3_csv(f).set_index('plus_code')
    wse = attrs.merge(wse_only, left_index=True, right_index=True)
    final_wses = pd.concat([wse, elev], join='inner', axis=1)
    if TRI:
        final_wses.to_csv(os.path.join(wse_dir, 'WSE_' + os.path.basename(f).replace('.', TRI + '.')))
        print('{} Complete'.format(os.path.basename(f).replace('.', TRI + '.')))
    else:
        final_wses.to_csv(os.path.join(wse_dir, 'WSE_' + os.path.basename(f)))
        print('{} Complete'.format(os.path.basename(f)))
          
print("\nOutputs Saved to {}".format(wse_dir))

# END